## wgMLST for mbovis

* http://www3.ridom.de/seqsphere/cgmlst/
* https://www.cgmlst.org/ncs/schema/741110/
* https://jcm.asm.org/content/52/7/2479
* https://www.biorxiv.org/content/10.1101/172858v1.full
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3980634/

## steps

* get MLST gene list
* assembly
* annotate
* assign allele numbers?

In [29]:
import sys,os,shutil,subprocess
import glob
import random
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', 150)
import pylab as plt
import seaborn as sns
from Bio import SeqIO, Phylo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from io import StringIO
import pathogenie as pg
import snipgenie
import toytree, toyplot

In [2]:
meta = pd.read_csv('/storage/btbgenie/mbovis_ireland/all_ireland_samples.csv')
vcf_file='/storage/btbgenie/wicklow_results/filtered.vcf.gz'

In [3]:
def get_samples_vcf(vcf_file):
    cmd = 'bcftools query -l %s' %vcf_file
    tmp = subprocess.check_output(cmd, shell=True)
    return tmp.decode().split('\n')
    
samplenames = get_samples_vcf(vcf_file)

In [4]:
path = '/storage/btbgenie/mbovis_ireland/Wicklow/'
files = glob.glob(os.path.join(path,'**', '*.fastq.gz'),recursive=True)
samp = snipgenie.get_samples(files,sep='_')
pairs = pd.pivot_table(samp,index=['sample'],columns=['pair'],values='filename',aggfunc='first').reset_index()
print (pairs[:2])

pair    sample                                                                                             1  \
0         1034                  /storage/btbgenie/mbovis_ireland/Wicklow/dog/1034_S91_L001-4_R2_001.fastq.gz   
1     13-11594  /storage/btbgenie/mbovis_ireland/Wicklow/Fastqs_15-03-19/13-11594_S85_L001-4_R1_001.fastq.gz   

pair                                                                                             2  
0                     /storage/btbgenie/mbovis_ireland/Wicklow/dog/1034_S91_L001-4_R1_001.fastq.gz  
1     /storage/btbgenie/mbovis_ireland/Wicklow/Fastqs_15-03-19/13-11594_S85_L001-4_R2_001.fastq.gz  


## create reference proteins for MLST

In [6]:
def make_reference_proteins():
    prots = pg.tools.genbank_to_dataframe('Mbovis-AF212297.2.gb',cds=True)
    prots = prots.fillna('')
    prots = prots.dropna(subset=['locus_tag'])
    ref_proteins = 'Mbovis_AF212297_proteins.fa'
    #get prokka type header for using in annotation
    prots['header'] = prots.apply(lambda x: '~~~'.join([x.locus_tag,x.gene,x['product'],'none']),1)
    pg.tools.dataframe_to_fasta(prots,idkey='header',outfile=ref_proteins)
    print (len(prots))
    return prots

prots=make_reference_proteins()

3994


In [5]:
def get_nucleotide_sequences(gb_file,out_file,idkey='locus_tag'):
    """protein nucleotide seqs from genbank"""
    
    recs = SeqIO.to_dict(SeqIO.parse(gb_file,'genbank'))
    chroms = list(recs.keys())
    result = []
    for chrom in chroms[:10]:       
        rec = recs[chrom]        
        for f in rec.features[1:]:
            q=f.qualifiers
            if f.type != 'CDS':
                continue
            seq = rec.seq[f.location.start:f.location.end]
            try:         
                new = SeqRecord(seq,id=q[idkey][0])
                result.append(new)
            except:
                #print (q)
                pass
    SeqIO.write(result,out_file,format='fasta')
    return result

res=get_nucleotide_sequences('Mbovis-AF212297.2.gb','Mbovis_AF212297_nuc.fa')

## assembly

In [ ]:
def spades(file1, file2, path, outfile=None, threads=4):
    """Run spades"""
    
    cmd = 'spades -t %s --pe1-1 %s --pe1-2 %s --careful -o %s' %(threads,file1,file2,path)    
    if not os.path.exists(path): 
        print (cmd)
        subprocess.check_output(cmd, shell=True)
    if outfile != None:
        shutil.copy(os.path.join(path,'scaffolds.fasta'),outfile)
    return outfile

assembly_path = '/storage/btbgenie/assembly/'

for i,r in pairs[5:10].iterrows():    
    name=r['sample']
    print (name)
    out = os.path.join(assembly_path,name+'.fa')
    print (out)
    spades(r[1], r[2], os.path.join(assembly_path,name), outfile=out, threads=12)

## get consensus sequence from alignment instead of assembly

In [7]:
def get_consensus(vcf_file, sample, out_file='consensus.fa'):
    """Get consensus sequence from vcf"""
      
    cmd='bcftools index -f %s' %vcf_file
    subprocess.check_output(cmd, shell=True)   
    cmd='cat {r} | bcftools consensus -s {s} {v} > {o}'.format(r=snipgenie.app.mbovis_genome,v=vcf_file,s=sample,o=out_file)
    #print (cmd)
    subprocess.check_output(cmd, shell=True)
    return

get_consensus(vcf_file, '1034','consensus.fa')


## annotate


In [ ]:
seqfile='/storage/btbgenie/assembly/1034.fa'
#seqfile='consensus.fa'
featdf,recs = pg.run_annotation(seqfile,
                                threads=10, kingdom='bacteria', trusted=ref_proteins)
outfile='/storage/btbgenie/annotation_assembly/1034.fa'

#write out nucl sequences
SeqIO.write(recs,'temp.gb','genbank')
res = get_nucleotide_sequences('temp.gb',outfile,idkey='protein_id')

In [853]:
rec=res[3]
print (rec.id)
rec.seq.translate()

Mb3672c


Seq('MPQGTVKWFNAEKGFGFIAPEDGSADVFVHYTEIQGTGFRTLEENQKVEFEIGH...SL*')

# initialise MLST table from reference genome

## filter only Mb proteins in scheme

In [27]:
mlst=pd.read_csv('mlst_scheme.csv')
mlst=mlst[mlst['Ridom Usage']=='MLST+']
targets=list(mlst.Target)

In [28]:
mbmap=pd.read_csv('final_mbovis_mapping.csv')
mbmap = mbmap[mbmap.Rv_tag.isin(targets)]
mb_tags = list(mbmap.Mb_tag)
prots = prots[prots.locus_tag.isin(mb_tags)]
nucseqs = pg.tools.fasta_to_dataframe('Mbovis_AF212297_nuc.fa')
nucseqs = nucseqs[nucseqs.name.isin(mb_tags)]

In [29]:
cols = ['name','allele','sequence']
ref = nucseqs.copy()
ref['allele'] = 1
ref=ref.rename(columns={'translation':'sequence'})
ref[cols].to_csv('mlst_db.csv.gz',index=False,compression='gzip')
print (len(ref))

3184


In [13]:
fastafile='/storage/btbgenie/annotation_assembly/1034.fa'
df = pg.tools.fasta_to_dataframe(fastafile)
df[df.name=='Mb0014c']

,name,sequence,description,type
3240,Mb0014c,CTACTGGCCGAACCTCAGCGTGATGATGCCGTCCCGGTTGACGCCGGTCCCCGCCGGCGGGTTTTGATAGACGACCCGGTTGTGTTGGGAGCCACCGGCGTCGACGTCGGCCCCTTTGTCGAGCATCCCGGTCCAGCCCAGCGCGC...,Mb0014c <unknown description>,CDS


In [8]:
def find_alleles(fastafile):
    """Find allele by simple matches to the reference table of known sequences.
    Returns:
        dataframe with allele number for each gene
        dataframe with new alleles to add to db
    """
    
    db = pd.read_csv('mlst_db.csv.gz')
    names = db.name.unique()
    df = pg.tools.fasta_to_dataframe(fastafile).reset_index()
    #print (df.iloc[0])
    result=[]
    new=[]
    for name in names:
        #print (name)
        s = db[db.name==name]
        gene = df[df.name==name]
        #print (gene)
        if len(gene)==0:
            #print (name)
            #missing gene in target
            result.append((name,0))
            continue
        target = gene.iloc[0].sequence
        found = s[s.sequence==target]        
        #print (target,found)
        if len(found)>0:
            found = found.iloc[0]
            result.append((name,found.allele))
        else:
            #assign new allele            
            newallele = s.allele.max()+1
            result.append((name,newallele))
            new.append([name,newallele,target])
    prof = pd.DataFrame(result,columns=['name','allele'])
    prof['allele'] = prof.allele.astype(int)
    #new additions
    new = pd.DataFrame(new,columns=['name','allele','sequence'])
    return prof, new
    
def update_mlst_db(new):
    """Update the database of MLST profiles"""
    
    db = pd.read_csv('mlst_db.csv.gz')
    db = pd.concat([db,new])
    db.to_csv('mlst_db.csv.gz', index=False, compression='gzip')
    print ('added %s new alleles' %len(new))
    return

fastafile='26-Mbovis.fa'
#prof,new = find_alleles(fastafile)
#update_mlst_db(new)
#print (prof[:100])

In [25]:
def type_sample(fastafile, outfile, threads=4, overwrite=False):
    """Type a single sample using wgMLST.
    Args:
        fastafile: fasta file to type from assembly or other
        
        path: output folder for annotations
    Returns:
        dataframe of MLST profile
    """
    
    ref_proteins = 'Mbovis_AF212297_proteins.fa'
    if overwrite == True or not os.path.exists(outfile):
        #annotate
        featdf,recs = pg.run_annotation(fastafile, threads=threads,
                                        kingdom='bacteria', trusted=ref_proteins)
        #get nucl sequences from annotation    
        SeqIO.write(recs,'temp.gb','genbank')
        get_nucleotide_sequences('temp.gb',outfile,idkey='protein_id')
    
    #find alleles
    res,new = find_alleles(outfile)
    #print (res)
    #update db
    update_mlst_db(new)
    return res

get_consensus(vcf_file, '26-MBovis')
fastafile = 'consensus.fa'
res = type_sample(fastafile, '26-Mbovis.fa', threads=10)

added 0 new alleles


In [10]:
def get_profile_string(df):
    return ''.join(df.allele.astype(str))

## profiles from assembly - not needed?

In [ ]:
profs = {}
omit=['182-MBovis','19-4281','19-2438']
samples=glob.glob(os.path.join(assembly_path,'*.fa'))
pdf=pd.DataFrame()
for s in samples:
    print (s)
    sample = os.path.splitext(os.path.basename(s))[0]    
    if sample in omit:
        continue
    profile = type_sample(s, '/storage/btbgenie/annotation_assembly/%s.fa' %sample, threads=12)
    pdf[sample] = profile['allele']
    profs[sample] = get_profile_string(profile)

## profiles from consensus seqs

In [11]:
def run_samples(vcf_file, outdir, omit=[], **kwargs):
    """Run samples in a vcf file.
    Args:
        vcf_file: multi sample variant file from previous calling
        outdir: folder for writing intermediate files
    Returns:
        dict of mst profiles
    """

    profs = {}
    samplenames = get_samples_vcf(vcf_file)
    print (samplenames)
    for s in samplenames[:15]:
        print (s)
        if s in omit or s =='':
            continue
        get_consensus(vcf_file, s)
        outfile = os.path.join(outdir, '%s.fa' %s)
        profile = type_sample('consensus.fa', outfile, **kwargs)
        profs[s] = list(profile.allele)
    return profs

## find distances between mlst profiles

In [12]:
def diff_profiles(s1, s2):    
    return sum(1 for a, b in zip(s1, s2) if a != b)

In [13]:
def dist_matrix(profiles):
    """Distance matrix of a set of profiles"""
    
    dist=[]
    for s in profiles:
        x=profiles[s]
        row=[]
        for s in profiles:
            d = diff_profiles(x,profiles[s])
            row.append(d)
        dist.append(row)
    D = pd.DataFrame(dist,columns=profiles.keys(),index=profiles.keys())
    return D

In [14]:
def tree_from_distmatrix(D, treefile):
    
    from skbio import DistanceMatrix
    from skbio.tree import nj
    ids = list(D.index)
    dm = DistanceMatrix(D.values, ids)
    tree = nj(dm)
    #print(tree.ascii_art())
    tree.write(treefile, 'newick')
    return tree

In [ ]:
import random
test = {'A':'1112233','B':'1112213','C':'2111213','D':'2111232','E':'212211','F':'2111222'}

d = dist_matrix(test)
tree_from_distmatrix(d)
tre = toytree.tree(treefile)
canvas,t,r=tre.draw(layout='r',scalebar=True,width=500)
sns.clustermap(d,figsize=(5,5))

## test on wicklow samples

In [ ]:
omit=['182-MBovis','19-4281','19-2438']
profs = run_samples(vcf_file, '/storage/btbgenie/annotation_consensus', threads=12, omit=omit)
for i in profs:
    print (i,len(profs[i]))

In [57]:
D = dist_matrix(profs)
D.to_csv('dist_mlst.csv',index=False)
#sns.clustermap(D,xticklabels=True,yticklabels=True,figsize=(6,6))
treefile='temp.newick'
tree_from_distmatrix(D, treefile)
tre = toytree.tree(treefile)
tre=tre.root('43-MBovis')
canvas,t,r=tre.draw(layout='r',scalebar=True,height=400,width=500)
import toyplot.png
#toyplot.png.render(canvas, "mlst-tree.png")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="400.0px" viewBox="0 0 500.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t76a87b1d39c6487899583756d17cca82"> 42-MBovis 59-MBovis 37-MBovis 38-MBovis 23-MBovis 25-MBovis 39-MBovis 49-MBovis 1034 50-MBovis 36-MBovis 24-MBovis 29-MBovis 43-MBovis 0 8 16 24 31

In [58]:
tre2 = toytree.tree('/storage/btbgenie/wicklow_results/tree.newick')
drop = list(set(tre2.get_tip_labels()) - set(tre.get_tip_labels()))
tre2=tre2.drop_tips(drop)
tre2=tre2.root('43-MBovis')
canvas,t,r=tre2.draw(layout='r',scalebar=True,height=400,width=500)
#toyplot.png.render(canvas, "snp-tree.png")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="400.0px" viewBox="0 0 500.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta80dac38d5244d8b9fb1d41ae0c73b62"> 49-MBovis 39-MBovis 23-MBovis 25-MBovis 38-MBovis 42-MBovis 59-MBovis 37-MBovis 36-MBovis 50-MBovis 24-MBovis 29-MBovis 1034 43-MBovis 0 9 19 28 37

In [36]:
import dendropy
from dendropy.calculate import treecompare
tns = dendropy.TaxonNamespace()
tree1 = dendropy.Tree.get(path=treefile, schema='newick',taxon_namespace=tns)
tree2 = dendropy.Tree.get(path='/storage/btbgenie/wicklow_results/RAxML_bestTree.variants', schema='newick',taxon_namespace=tns)


In [37]:
print(treecompare.euclidean_distance(tree1, tree2))

36.61389000303188


# sensitivity analysis - SNPs vs MLST

## phastsim

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7987011/

In [65]:
refseq = SeqIO.read('Mbovis_AF212297.fa','fasta')
scale = (1/len(refseq))*4
#scale=8e-6
cmd = 'phastSim --outpath simulation_output/ --seed 1 --createFasta' \
         ' --createPhylip --treeFile sim.newick' \
         ' --scale {s} --invariable .1 --alpha 1.0 --omegaAlpha 1.0' \
         ' --reference Mbovis_AF212297.fa'.format(s=scale)
cmd

'phastSim --outpath simulation_output/ --seed 1 --createFasta --createPhylip --treeFile sim.newick --scale 9.195605236345446e-07 --invariable .1 --alpha 1.0 --omegaAlpha 1.0 --reference Mbovis_AF212297.fa'

In [16]:
simrecs = list(SeqIO.parse('sim_seqs.fa','fasta'))
diff_profiles(simrecs[0].seq,simrecs[1].seq)

75

## make fastqs from sequences

In [17]:
def make_fastq(ref, outfile, cmp=100):
    
    f1 = '/storage/btbgenie/mbovis_ireland/Wicklow/Fastqs_07-01-18/17-MBovis_S21_L001-4_R1_001.fastq.gz'
    f2 = '/storage/btbgenie/mbovis_ireland/Wicklow/Fastqs_07-01-18/17-MBovis_S21_L001-4_R2_001.fastq.gz'
    cmd = 'java -jar /local/ArtificialFastqGenerator/ArtificialFastqGenerator.jar -O {o}'\
          ' -R {r} -S ">temp" -RL 150 -CMP {cmp} -CSD 0.2 -SE true'.format(r=ref, o=outfile,cmp=cmp,f1=f1,f2=f2) #-URQS true -F1 {f1} -F2 {f2}
    print (cmd)
    subprocess.check_output(cmd, shell=True)
    return

In [66]:
def generate_fastqs(infile, outpath):
    
    from joblib import Parallel, delayed
    import multiprocessing, time
    num_cores = 4
    
    simrecs = list(SeqIO.parse(infile,'fasta'))
    def my_func(rec):
        from tempfile import mkstemp
        x,tmp = mkstemp()
        SeqIO.write(SeqRecord(rec.seq,id='temp'), tmp, 'fasta')
        out = os.path.join(outpath,rec.id)
        make_fastq(tmp, out)

    st = time.time()
    Parallel(n_jobs=num_cores)(delayed(my_func)(i) for i in simrecs)
    print (time.time()-st)
    cmd = 'pigz %s/*.fastq' %outpath
    subprocess.check_output(cmd, shell=True)
    
generate_fastqs('sim_seqs.fa', 'sim_fastq')
#generate_fastqs('sim_seqs1.fa', 'sim_fastq1')

550.2393217086792


## run snipgenie on simulated fastqs

In [ ]:
import snipgenie
args = {'threads':10, 'outdir': 'sim_results', 'labelsep':'_',
        'input':['sim_fastq/'], 'overwrite':True,
         'species':'Mbovis-AF212297',        
         'custom_filters': True,       
         'buildtree':True}
W = snipgenie.app.WorkFlow(**args)
st = W.setup()
W.run()

In [68]:
tresnps = toytree.tree('sim_results/tree.newick')
tresnps=tresnps.root('1')
tresnps=tresnps.drop_tips('ref')
canvas,t,r=tresnps.draw(layout='r',scalebar=True,height=400,width=500,tip_labels_style={"font-size":"19px"})
sd=snpdist = pd.read_csv('sim_results/snpdist.csv',index_col=0).iloc[1:,1:]
print (sd)
canvas.text(300, 20, "SNP", style={"font-size":"32px"});
import toyplot.png
#toyplot.png.render(canvas, "snp_tree_sim.png")

    4   7   3   1   9   2   8   6   5
4   0  21  15  12  19  18  19  19   9
7  21   0  22  17  26  23  26  24  16
3  15  22   0  13   4  19   8  20  10
1  12  17  13   0  17   8  17   7   7
9  19  26   4  17   0  23  12  24  14
2  18  23  19   8  23   0  23  15  13
8  19  26   8  17  12  23   0  24  14
6  19  24  20   7  24  15  24   0  14
5   9  16  10   7  14  13  14  14   0


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="400.0px" viewBox="0 0 500.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te2b59847ff42431a9c73428bdc1a2369"> 3 9 8 4 5 7 2 6 1 0 6 13 19 25 SNP

In [62]:
tresboot = toytree.mtree('sim_results/RAxML_bootstrap.variants')
canvas,t,r=tresboot.draw(layout='r', nrows=2, ncols=5,height=400,width=700)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="400.0px" viewBox="0 0 700.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5a28173ff3ef4674ba71cea5e1092838"> 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7 3 9 8 5 4 6 1 2 ref 7

## mlst on simulated fastqs

In [45]:
sim_vcf = 'sim_results/filtered.vcf.gz'
simprofs = run_samples(sim_vcf, 'sim_mlst', threads=12)

done
added 19 new alleles



In [64]:
D = dist_matrix(simprofs)
print (D)
D.to_csv('dist_mlst.csv')
treefile='mlst.newick'
tree_from_distmatrix(D, treefile)
tremlst = toytree.tree(treefile)
#tremlst=tremlst.root('1')
canvas,t,r=tremlst.draw(layout='r',scalebar=True,height=400,width=500,tip_labels_style={"font-size":"19px"})
canvas.text(300, 20, "wgMLST", style={"font-size":"32px"});
#toyplot.png.render(canvas, "wgmlst_tree_sim.png")

     4    7    3    1    9    2    8    6    5
4    0  186  114  122  141  179  147  157   63
7  186    0  168  140  191  197  197  177  166
3  114  168    0  100   38  162   66  136   91
1  122  140  100    0  125   75  135   48   99
9  141  191   38  125    0  187   96  160  119
2  179  197  162   75  187    0  192  115  156
8  147  197   66  135   96  192    0  169  124
6  157  177  136   48  160  115  169    0  136
5   63  166   91   99  119  156  124  136    0


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="400.0px" viewBox="0 0 500.0 400.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t229a37e04f8d424195dc005956fc69c7"> 3 9 8 4 5 7 2 6 1 0 34 67 101 134 wgMLST